# Exploratory Data & Business Analysis
The **Data Science Lifecycle** includes steps to perform Exploratory Data Analysis as well as evaluate the outputs of machine learning models. Since most data science projects in industry are set to solve specific business objectives, it's important to evaluate the business impact of the models we build. 

### Exploratory Data Analysis (EDA)
[Exploratory Data Analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis) is the process of analyzing data to derive insights, often done using visualizations. While this process can be done manually, there are a variety of open source tools which can come in handy. We will explore the usage of [pandas-profiling](https://github.com/pandas-profiling/pandas-profiling) below. 

**Note:** All required Python packages required for this course are installed via `conda`. Please refer to [environment.yml](../environment.yml) for a detailed list of packages & versions used in this course.

In [1]:
import pandas as pd
import pandas_profiling
import numpy as np

In [2]:
airlines_df = pd.read_csv("../data/external/allyears2k.csv", encoding="ISO-8859-1", low_memory=False)
target = "IsDepDelayed"
print("Dataset has {} entries and {} features".format(*airlines_df.shape))

# preview data
airlines_df.head()

Dataset has 43978 entries and 31 features


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
0,1987,10,14,3,741.0,730,912.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,YES,YES
1,1987,10,15,4,729.0,730,903.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,YES,NO
2,1987,10,17,6,741.0,730,918.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,YES,YES
3,1987,10,18,7,729.0,730,847.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NO,NO
4,1987,10,19,1,749.0,730,922.0,849,PS,1451,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,YES,YES


`pandas-profiling` can automatically generate a report with the following features:

* Essentials: type, unique values, missing values
* Quantile statistics like minimum value, Q1, median, Q3, maximum, range, interquartile range
* Descriptive statistics like mean, mode, standard deviation, sum, median absolute deviation, coefficient of variation, kurtosis, skewness
* Most frequent values
* Histograms
* Correlations highlighting of highly correlated variables, Spearman and Pearson matrixes

Here's how...

In [3]:
pandas_profiling.ProfileReport(airlines_df)

Number of variables,31
Number of observations,43978
Total Missing (%),20.0%
Total size in memory,10.4 MiB
Average record size in memory,248.0 B
Numeric,17
Categorical,7
Boolean,3
Date,0
Text (Unique),0
Rejected,4


+ Gather insights using exploratory methods, descriptive & inferential statistics
    + Find median, mode, std dev, min, max, average for each column. Do these make sense in the context of the column?
    + Do values have reasonable lower & upper bounds?
    + Univariate feature distributions (to observe stability & other patterns of a given feature like skew)
    + Feature & target correlations
    + Target analysis (plot of feature vs target)
    + Are there any outliers?
    + Do the column values seem to follow a normal distribution? Uniform? Exponential (i.e. long tail)?
    + Ensure correct data types are imputed
    + Look at the values. Ensure they make sense in the context of each column
    + Look for missing/empty values
    + Determine if there is a target imbalance? One class is represented less 15% of the outcomes. 
    + For categorical fields, what are the unique values in the field?
    + For numeric fields, are all values numbers?

If you want to generate a HTML report file, save the ProfileReport to an object and use the `to_file()` function. You can see a HTML version of our data located [here](../data/processed/data_analysis.html).

In [ ]:
profile = pandas_profiling.ProfileReport(airlines_df)
profile.to_file(outputfile="../data/processed/data_analysis.html")

When we are building machine learning models, several learners are sensitive to features which are highly correlated. In other words, including features with high correlations reduces the predictive power of the model. `pandas-profiling` can automatically retrieve the list of variables which are rejected due to high correlations... 

In [ ]:
profile = pandas_profiling.ProfileReport(airlines_df)
rejected_variables = profile.get_rejected_variables(threshold=0.9)
rejected_variables

It's of no surprise that **Scheduled Elapsed Time** is positively correlated to **Distance** as further distances take longer time to travel; thus resulting in longer time spent in the air.

### Business Analysis
Before a model is released into production, we must understand how the outputs of the machine learning model **impact the business objectives**. One way to assess the impact to the business objective is to define the cost of errors (bad predictions). In the case of a binary classifier, the two types of errors which can result are **Type 1 (False Positive)** and **Type 2 (False Negative)**.

Consider the following confusion matrix for the airline dataset we've been using in this course...
  
![alt text](../assets/images/Flight_Confusion_Matrix.png)

Let's refer to the test set confusion matrix which resulted from the model development process covered earlier.

```
Confusion Matrix...
[[3768 3178]
 [2009 5558]]
```

 ```
 True Positive (TP, correct prediction): 3,768. 
 Assume flight on-time results in $100 of revenue per customer.
 
 True Negative (TN, correct prediction): 5,558
 Assume flight which has been delayed results in $100 of revenue per customer.

 False Positive (FP, incorrect prediction): 3,178
 Assume the inconvenience of a delayed flight when the customer was notified it will be on-time is $500 of cost per customer.
 
 False Negative (FN, incorrect prediction): 2,009
 Assume the inconvenience of a flight on-time when the customer was notified it will be delayed is $100 of cost per customer.

```

One approach to evaluating the business result of the predictions from this model is using the following formula:


`Business Impact = Revenue * TP + Revenue * TN - Cost * FP - Cost * FN`

In [19]:
business_impact = 700 * 3768 + 700 * 5558 - 3178 * 500 - 2009 * 100
business_impact

4738300

This model will result in a net positive value of $4,738,300 if we assume a single passenger was to travel on every flight in our test set.